In [ ]:
import IPython.display as ipd
import librosa.display
import numpy as np
import pandas as pd
from datetime import datetime
import tensorflow as tf

# Sources:
# https://www.youtube.com/watch?v=WJI-17MNpdE

In [ ]:
start_time = datetime.now()

In [ ]:
df = pd.read_csv('Dataset/participant_urbansound8k.csv')

In [ ]:
labels = df.iloc[:7079, 3]
files = df.iloc[:, 0]
folder = df.iloc[:, 2]

In [ ]:
tf.random.set_seed(310503)
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[13, 1000]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10000, activation="relu"))
model.add(tf.keras.layers.Dense(5000, activation="relu"))
model.add(tf.keras.layers.Dense(3512, activation="relu"))
model.add(tf.keras.layers.Dense(2312, activation="relu"))
model.add(tf.keras.layers.Dense(1512, activation="relu"))
model.add(tf.keras.layers.Dense(1055, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

data = []

for row in range(10000): #ICI QUON LOOP A TRAVERS LES 10 000 DONNÉES (REMPLACER 1 DANS RANGE QUAND ÇA MARCHE BIEN)
    try:
        audio_file = f"Dataset/fold{folder[row]}/{files[row]}"
        audio = ipd.Audio(audio_file)
        if librosa.get_duration(filename=audio_file) != 4.0:
            labels.pop(row)
            continue
        signal, sr = librosa.load(audio_file)
        mfccs = librosa.feature.mfcc(y=signal, n_mfcc=13, sr=sr)
        deltaMfccs = librosa.feature.delta(mfccs)
        delta2Mfccs = librosa.feature.delta(mfccs, order=2)
        compmfccs = np.concatenate(np.asarray((abs(np.concatenate(mfccs)),abs(np.concatenate(deltaMfccs)),abs(np.concatenate(delta2Mfccs)))))
        compmfccs = NormalizeData(compmfccs)
        data.append(compmfccs)
    except:
        print(row)
        print(f"Dataset/fold{folder[row]}/{files[row]}")

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
labels = labels.astype(int)
X_train, Y_train = labels[:-2000], data[:-2000]
X_valid, Y_valid = labels[-2000:], data[-2000:]

In [ ]:
print(Y_train)

In [ ]:
history = model.fit(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))